In [ ]:
#colab環境にgdriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from keras.datasets import mnist
import cv2
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#-1~+1に正規化
X_train = (X_train.astype(np.float32) - 127.5)/127.5


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Reshape,Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import cv2

import matplotlib.pyplot as plt
import sys
import numpy as np




class DCGAN():
    def __init__(self):
        # Input shape
        self.input_img_rows = 28
        self.input_img_cols = 28
        self.input_channels = 1
        self.input_img_shape = (self.input_img_rows, self.input_img_cols, self.input_channels,)

        self.latent_dim = 100

        optimizer = Adam(0.00001, 0.5)

        # 2値分類のための識別モデルを作成
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        
        self.generator = self.build_generator()
        z = Input(shape = self.latent_dim)
        img = self.generator(z)

        # discriminatorのパラメータ固定
        self.discriminator.trainable = False
        valid = self.discriminator(img)

        self.combined = Model(z, valid)#並列結合
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):
        model = Sequential()
        model.add(Dense(512*7*7, input_shape=[self.latent_dim]))
        model.add(Reshape((7,7,512)))

        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(UpSampling2D((2,2)))

        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(UpSampling2D((2,2)))

        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))

        model.add(Dense(self.input_channels))
        model.add(Activation('tanh')) #-1 ~ 1 に分散してくれるやつ

        model.summary()

        img = Input(shape=self.latent_dim)
        validity = model(img)

        return Model(img,validity)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(512, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(256, kernel_size=3, strides=1, input_shape=self.input_img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        #model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(64, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.input_img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=500):
        print("実行1")
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        #コンスタントなノイズ
        imagegennoise = np.random.uniform(-1, 1, (32,self.latent_dim))

        for epoch in range(epochs):

            # 本物の画像をbatchの数だけランダムで持ってきます
            idx = np.random.randint(0, len(X_train), batch_size)
            true_imgs = X_train[idx]

            # batchの数だけノイズを生成し、generatorに画像を生成させる
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # 本物の画像とフェイク画像を識別機に学習させます。
            d_loss_real = self.discriminator.train_on_batch(true_imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # 誤差を伝搬させて、generatorを学習させます
            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % 50 == 0:
                testimg = self.generator.predict(imagegennoise)
                dst = np.array((testimg * 0.5 + 0.5) * 255, np.int32) #0~255(人間に見やすく)
                #32,28,28,1
                dst = dst.reshape(4,8,1,28,28)
                dst = dst.transpose(0,3,1,4,2)
                dst = dst.reshape(4*28,8*28,1)
                cv2.imwrite(f'/content/drive/MyDrive/GANstudy/2.12/createimages/test{str(epoch)}.jpg', dst)


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=100000, batch_size=128, save_interval=500)

Streaming output truncated to the last 5000 lines.
13147 [D loss: 0.691926, acc.: 51.17%] [G loss: 0.695999]
13148 [D loss: 0.692014, acc.: 55.08%] [G loss: 0.695305]
13149 [D loss: 0.692062, acc.: 53.12%] [G loss: 0.694632]
13150 [D loss: 0.690913, acc.: 54.69%] [G loss: 0.695111]
13151 [D loss: 0.690405, acc.: 59.77%] [G loss: 0.696541]
13152 [D loss: 0.692189, acc.: 55.08%] [G loss: 0.694260]
13153 [D loss: 0.691080, acc.: 60.16%] [G loss: 0.696036]
13154 [D loss: 0.692218, acc.: 51.17%] [G loss: 0.695436]
13155 [D loss: 0.693156, acc.: 48.05%] [G loss: 0.694312]
13156 [D loss: 0.691920, acc.: 54.69%] [G loss: 0.697861]
13157 [D loss: 0.691042, acc.: 58.59%] [G loss: 0.699924]
13158 [D loss: 0.690715, acc.: 60.55%] [G loss: 0.701901]
13159 [D loss: 0.690029, acc.: 60.55%] [G loss: 0.699356]
13160 [D loss: 0.690762, acc.: 58.59%] [G loss: 0.700242]
13161 [D loss: 0.692016, acc.: 53.91%] [G loss: 0.699879]
13162 [D loss: 0.691734, acc.: 53.12%] [G loss: 0.699172]
13163 [D loss: 0.6916

KeyboardInterrupt: ignored